<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dual-Numbers" data-toc-modified-id="Dual-Numbers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dual Numbers</a></span><ul class="toc-item"><li><span><a href="#Basic-Arithmetic" data-toc-modified-id="Basic-Arithmetic-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Basic Arithmetic</a></span><ul class="toc-item"><li><span><a href="#Addition" data-toc-modified-id="Addition-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Addition</a></span></li><li><span><a href="#Subtraction" data-toc-modified-id="Subtraction-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Subtraction</a></span></li><li><span><a href="#Multiplication" data-toc-modified-id="Multiplication-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Multiplication</a></span></li><li><span><a href="#Division" data-toc-modified-id="Division-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Division</a></span></li><li><span><a href="#Power" data-toc-modified-id="Power-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Power</a></span></li></ul></li></ul></li><li><span><a href="#Automatic-Differentation" data-toc-modified-id="Automatic-Differentation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Automatic Differentation</a></span><ul class="toc-item"><li><span><a href="#Observations" data-toc-modified-id="Observations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Observations</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Implementation</a></span></li></ul></li><li><span><a href="#Higher-Order-Derivatives" data-toc-modified-id="Higher-Order-Derivatives-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Higher-Order Derivatives</a></span><ul class="toc-item"><li><span><a href="#Basic-Arithmetic" data-toc-modified-id="Basic-Arithmetic-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Basic Arithmetic</a></span><ul class="toc-item"><li><span><a href="#Addition" data-toc-modified-id="Addition-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Addition</a></span></li><li><span><a href="#Subtraction" data-toc-modified-id="Subtraction-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Subtraction</a></span></li><li><span><a href="#Multiplication" data-toc-modified-id="Multiplication-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Multiplication</a></span></li><li><span><a href="#Division" data-toc-modified-id="Division-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Division</a></span></li><li><span><a href="#Power" data-toc-modified-id="Power-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Power</a></span></li></ul></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Implementation</a></span></li></ul></li><li><span><a href="#Further-Work" data-toc-modified-id="Further-Work-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Further Work</a></span></li></ul></div>

# Dual Numbers
Dual number of a real number $a$ is of the form $a+b\epsilon$ where $\epsilon$ is a very small number such that $\epsilon^2=0$, meaning that it is a very small number & $b$ is usually 1.

## Basic Arithmetic
### Addition
Adding two dual numbers is similar to adding two complex numbers, we deal with real and dual numbers seperately.
$$(3 + 4\epsilon) + (1 + 2\epsilon) = 4 + 6\epsilon$$

### Subtraction
$$(3 + 4\epsilon) - (1 + 2\epsilon) = 2 + 2\epsilon$$

### Multiplication
For multiplying two dual numbers, we use FOIL(Firsts, Outers, Inners, Lasts) techniques.
$$(3+4\epsilon)*(1+2\epsilon)$$
$$3+8\epsilon^2+4\epsilon+6\epsilon$$
$$3+10\epsilon+8\epsilon^2$$
since $\epsilon^2=0$
$$3+10\epsilon$$

Generalizing, $(a+b\epsilon)*(c+d\epsilon)$,results in - 
$$ac+(bc+ad)\epsilon$$

### Division
Dividing one dual number by other involves taking the conjugate of the divisor.
$$\frac{(3+4\epsilon)}{(1+2\epsilon)}$$
taking conjugates of the denominator
$$\frac{(3+4\epsilon)*(1-2\epsilon)}{(1+2\epsilon)*(1-2\epsilon)}$$

$$\frac{3+4\epsilon-6\epsilon-8\epsilon^2}{1-4\epsilon^2}$$
since $\epsilon^2=0$
$$\frac{(3-2\epsilon)}{1}$$
Seperating the numerator
$$3-(2\epsilon)$$


Essentially, if numerator is $(a+b\epsilon)$ and denominator is $(c+d\epsilon)$, then division results in - 
$$\frac{a}{c}+\frac{(bc-ad)}{c^2}\epsilon$$

### Power
$$\begin{align*}
(a+b\epsilon)^{c+d\epsilon} &= e^{ (c+d\epsilon)\log(a+b\epsilon) } 
  & a=e^{\log a} &;  \log a^n=n\log(a)\\
&=e^{ (c+d\epsilon)(\log a+\log(1+\frac{b}{a}\epsilon))} 
  & \log(a+b)=\log(a)+\log(1+\frac{b}{a})\\
&=e^{ (c+d\epsilon)(\log a+b\epsilon/a) } 
  & \log(1+a)=a\\
&=e^{ c\log a+\epsilon(d\log a+b c/a) }& (a+b\epsilon)(c+d\epsilon)=ac+(bc+ad)\epsilon\\
&=e^{c\log a+\epsilon d\log a}\,e^{ \epsilon \frac{b\,c}{a} }\\
&=e^{\log a (c+\epsilon d)}\,e^{ \epsilon \frac{b\,c}{a} } \\
&=(e^{\log a})^{(c+\epsilon d)}\,e^{ \epsilon \frac{b\,c}{a} }
    & a^{mn}=(a^{m})^{n}\\
&=a^{c+\epsilon d}\,e^{\epsilon \frac{b\,c}{a}}
    & e^{\log a}=a
\end{align*}$$

# Automatic Differentation

Automatic Differentiation allows you to calculate the derivative of a function while calculating the value of the function. Essentially, we get both the value $f(x)$ and $f'(x)$.

Taylor Series is - 
![TaylorSeries](https://wikimedia.org/api/rest_v1/media/math/render/svg/46dd2bf78ad7d792988cf616a6ea94024f30b3d9)

Instead of approximating $f$ at about a real number $a$, if we instead approximate it about it's dual number $a+\epsilon$, we end up with the expression -
![TaylorSeries](https://miro.medium.com/max/1052/1*dieW9KTLFmDG2hc7wzzKmw.gif)

Since $\epsilon^2=0$, all the terms after the second term end up being $0$. So, the expression simplifies to -
![automatic-differentiation](https://miro.medium.com/max/628/1*x6s9-ZrFy6sA1SRYit-4fw.gif)


While performing any binary operation between a real number and a dual number, we convert real number to it's dual form using 0 for the dual component. For example; calculating $3x$ at $x=4+1\epsilon$

Convert $3$ to it's dual form: $3+0\epsilon$.

$$(3+0\epsilon)*(4+1\epsilon)$$
$$12+0\epsilon+3\epsilon+0\epsilon^2$$
$$12+3\epsilon$$


Instead of calculating $f(a)$ if we calculate the function's value for it's dual number i.e. $f(a+b\epsilon)$, we end up getting $f(a)$ and $f'(a)$ both. For example, we will start with $f(x)=3x+2$ and calculate $f(4)$ and $f'(4)$.

* **Step1** - We convert our $4$ into a dual number, using $1$ for the dual component. It gives us $4+1\epsilon$
* **Step2** - Similarily, we convert the constant $3$ in $3x+4$ into a dual number, using $0$ for the dual component.
* **Step3** - Perform multiplication as did in previous section
$$(3+0\epsilon)*(4+1\epsilon)$$
$$12+0\epsilon+3\epsilon+0\epsilon^2$$
$$12+3\epsilon$$
* **Step4** - Lastly, we need to add the constant 2, using 0 again for the dual component since it’s just a constant.
$$(12+3\epsilon)+(2+0\epsilon)$$
$$14+3\epsilon$$
* **Step5** - In the result above the real component $14$ gives us $f(x=4)$ and the dual component gives $f'(x=4)$

This way we can calculate the value as well as the derivative of any function at any value by calculating it's value for the corresponding dual number.

## Observations

1. We have a completely mechanical way to evaluate derivatives at a point. **This mechanism works for any function that can be composed out of the primitive operations that we have extended to dual numbers.** 
2. It turns every arithmetic operation into a fixed number of new arithmetic operations, so the **arithmetic cost goes up by a small constant factor.** For example, addition requires 2 extra steps. Similarily, multiplication requires 3 additional steps and so on. 
3. We also do not introduce any gross numerical sins, so we can reasonably expect the accuracy of the derivative to be no worse than that of the primal computation.3

## Implementation

In [2]:
from scipy.misc import derivative

In [10]:
class DualNumber:
    def __init__(self, real, dual):
        self.real = real
        self.dual = dual

    def __add__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real + other.real, 
                              self.dual + other.dual)
        else:
            return DualNumber(self.real + other, self.dual)
    # addition is commutative
    __radd__ = __add__

    def __sub__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real - other.real,
                              self.dual - other.dual)
        else:
            return DualNumber(self.real - other, self.dual)

    def __rsub__(self, other):
        return DualNumber(other, 0) - self

    def __mul__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real * other.real,
                              self.real * other.dual + self.dual * other.real)
        else:
            return DualNumber(self.real * other, self.dual * other)
    # multiplication is commutative
    __rmul__ = __mul__

    def __div__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real/other.real,
                              (self.dual*other.real - self.real*other.dual)/(other.real**2))
        else:
            return (1/other) * self

    def __rtruediv__(self, other):
        return DualNumber(other, 0).__div__(self)

    def __pow__(self, other):
        return DualNumber(self.real**other,
                          self.dual * other * self.real**(other - 1))

    def __repr__(self):
        return repr(self.real) + ' + ' + repr(self.dual) + '*epsilon'

In [11]:
def auto_diff(f, x):
    return f(DualNumber(x, 1.)).dual

$$f(x)=1/x^5$$

In [27]:
x = 0.1
f = lambda x: 1./x**5
print(f"[AUTODIFF] f'({x}): {auto_diff(f, x):.4f}") 
print(f"[SCIPY] f'({x}): {derivative(f, x0=x, n=1, dx=1e-6):.4f}")

[AUTODIFF] f'(0.1): -5000000.0000
[SCIPY] f'(0.1): -5000000.0035


$$f(x)=5x^2+4x+1$$

In [26]:
x = 2
f = lambda x: 5*(x**2)+4*x+1
print(f"[AUTODIFF] f'({x}): {auto_diff(f, x):.4f}") 
print(f"[SCIPY] f'({x}): {derivative(f, x0=x, n=1, dx=1e-6):.4f}")

[AUTODIFF] f'(2): 24.0000
[SCIPY] f'(2): 24.0000


# Higher-Order Derivatives


How to extend this to say calculating second or third derivatives? Does automatic differentation hold for higher order derivatives? If yes, then what changes? For example, I read somewhere that for calculating second-order derivative, we have to make sure that $\epsilon^3=0$ instead of $\epsilon^2$. 

If this holds true, then we can expand the derivation in the previous section, by adding yet another term $\frac{f''(x)}{2!}\epsilon^2$

![AutomaticDifferentiation](http://latex2png.com/pngs/ccfe8be65fc072f3ac563949c27bdba9.png)

In this scenario, ***Addition & Subtraction*** still remain unchanged as they do not contain any squared dual components. However, ***Multiplication & Divison*** change.

Let $\epsilon^3=10^{-12}$, a really small number. Then $\epsilon^2\approx=10^{-8}$ which is still relatively small & will usually have any effect when computated w.r.t any other $\epsilon^2$ term only. Otherwise, I guess, we can safely ignore it when evaluated with real numbers only.

In [12]:
epsilon_cube = 1e-12

In [13]:
epsilon = np.power(epsilon_cube, 1/3)
epsilon_sqr = np.power(epsilon, 2)

In [20]:
print(f"ε^3: {epsilon_cube:1}")
print(f"ε^2: {epsilon_sqr:1e}")
print(f"ε: {epsilon:1e}")

ε^3: 1e-12
ε^2: 1.000000e-08
ε: 1.000000e-04


## Basic Arithmetic

### Addition
$$(a+b\epsilon+c\epsilon^2)+(x+y\epsilon+z\epsilon^2)=(a+x)+(b+y)\epsilon+(c+z)\epsilon^2$$

### Subtraction
$$(a+b\epsilon+c\epsilon^2)-(x+y\epsilon+z\epsilon^2)=(a-x)+(b-y)\epsilon+(c-z)\epsilon^2$$

### Multiplication
$(a+b\epsilon+c\epsilon^2)*(x+y\epsilon+z\epsilon^2)$, results in - 
$$ax+(ay+bx)\epsilon+(az+by+cx)\epsilon^2$$

### Division
$$\frac{(a+b\epsilon+c\epsilon^2)}{(x+y\epsilon+z\epsilon^2)}$$

assume, $x$ and $\epsilon(y+z\epsilon)$ as different terms and then apply conjugates

$$\frac{(a+b\epsilon+c\epsilon^2)*(x-\epsilon(y+z\epsilon))}{(x+\epsilon(y+z\epsilon))*(x-\epsilon(y+z\epsilon))}$$

$$\frac{ax+(bx-ay)\epsilon-(az+by+cx)\epsilon^2}{x^2-y^2\epsilon^2}$$

as per the comments in above section, we can ignore $\epsilon^2$ term in the denominator for the first two terms

$$\frac{a}{x}+\frac{(bx-ay)}{x^2}\epsilon-\frac{(az+by+cx)}{x^2-y^2\epsilon^2}\epsilon^2$$

### Power

## Implementation

In [ ]:
class DualNumber:
    def __init__(self, real, dual, dual_sq):
        self.real = real
        self.dual = dual
        self.dual_sq = dual_sq

    def __add__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real+other.real, 
                              self.dual+other.dual, 
                              self.dual_sq+other.dual_sq)
        else:
            return DualNumber(self.real+other, self.dual, self.dual_sq)

    # addition is commutative
    __radd__ = __add__

    def __sub__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real-other.real,
                              self.dual-other.dual, 
                              self.dual_sq-other.dual_sq)
        else:
            return DualNumber(self.real-other, self.dual, self.dual_sq)

    def __rsub__(self, other):
        return DualNumber(other, 0, 0) - self

    def __mul__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real*other.real,
                              self.real*other.dual+self.dual*other.real, 
                              self.real*other.dual_sq+self.dual*other.dual+self.dual_sq*other.real)
        else:
            return DualNumber(self.real*other, 
                              self.dual*other, 
                              self.dual_sq*other)
    # multiplication is commutative
    __rmul__ = __mul__

    def __div__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real/other.real,
                              (self.dual*other.real - self.real*other.dual)/(other.real**2))
        else:
            return (1/other) * self

    def __rtruediv__(self, other):
        return DualNumber(other, 0).__div__(self)

    def __pow__(self, other):
        return DualNumber(self.real**other,
                          self.dual * other * self.real**(other - 1))

    def __repr__(self):
        return repr(self.real) + ' + ' + repr(self.dual) + '*epsilon'

# Further Work
**Point 1**

It's clear that since the other numbers(5,3) are just constant in say $5x+3$, we consider 0 for their *dual component*. But, what is the thing about choosing $1$ for *dual components* while calculating $f(x)$ as $f(x+1\epsilon)$? 

$$\left. \frac{df(x)}{dx}\right|_x = \textrm{epsilon-coefficient}(\textrm{dual-version}(f)(x+1\epsilon)).$$

<br><br><br>

**Point 2**
As far as I could think of, automatic differentiation worked becuase all of the above functions were composed of basic primary operations. How would things work if something like non-linear functions or arbitrary functions jump in? I mean, Taylor series gives a good approximation of $sin(x), cos(x)$ or any function at any point for that matter, so will we have to integrate them as some sort of class method in the above class? Unclear about it at this point.

